# 0. Notebook Overview

### HR Employee Dataset – Data Cleaning (Raw → Cleaned)
This notebook focuses on transforming raw HR data into an analysis-ready dataset.

- Input: `employee_data.csv` (raw HR dataset)
- Output: `cleaned_employee_data.csv` (analysis-ready dataset)

This notebook covers:
1. Inspecting raw data quality (missing values, data types, duplicates)
2. Cleaning and standardizing key fields (dates, categories)
3. Fixing inconsistencies between employee status and termination information
4. Handling missing, incorrect, or outlier values
5. Creating core derived fields (IsActive, AttritionFlag, TenureDays)
6. Exporting a cleaned dataset for HR Workforce analysis (Notebook 2)

# 1. Import Libraries & Load Raw Data

In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

# Load raw dataset
df_raw = pd.read_csv("/kaggle/input/employeedataset/employee_data.csv")
df_raw.head()

# Fixed reference date for all time-based calculations
analysis_date = pd.Timestamp("2025-11-26")

### 📌 Explanation
To ensure reproducibility, this notebook uses a fixed **analysis date** to keep all time-based calculations (age, tenure, attrition) consistent. Using a static date prevents results from changing each time the notebook is rerun.

# 2. Inspect Basic Structure

In [2]:
# Basic overview of raw dataset
print("Shape (rows, cols):", df_raw.shape)
print("\nData Types:")
df_raw.info()

print("\nMissing Values:")
print(df_raw.isnull().sum().sort_values(ascending=False))

print("\nDuplicate Rows:", df_raw.duplicated().sum())

Shape (rows, cols): (3000, 26)

Data Types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   EmpID                       3000 non-null   int64 
 1   FirstName                   3000 non-null   object
 2   LastName                    3000 non-null   object
 3   StartDate                   3000 non-null   object
 4   ExitDate                    1533 non-null   object
 5   Title                       3000 non-null   object
 6   Supervisor                  3000 non-null   object
 7   ADEmail                     3000 non-null   object
 8   BusinessUnit                3000 non-null   object
 9   EmployeeStatus              3000 non-null   object
 10  EmployeeType                3000 non-null   object
 11  PayZone                     3000 non-null   object
 12  EmployeeClassificationType  3000 non-null   object
 13  Term

### 📌 Key Findings
- Dataset contains **3,000 rows × 26 columns** (employee-level records), which is an adequate size for workforce-level exploratory analysis.
- Several object columns should be converted to **categorical** types for cleaner analysis and better memory efficiency.
- **Date fields** (StartDate, ExitDate, DOB) are stored as strings and need conversion to `datetime` for accurate tenure, hiring, and attrition calculations.
- Missing ExitDate and TerminationDescription values align with active employees, which is expected.
- No duplicate rows were found.

# 3. Create Working Copy

To keep the raw dataset intact, I created a working copy (`df`) and selected only the fields needed for workforce, headcount, and attrition analysis.

### 🗂️ Column Classification
- **Required columns**: Core HR fields used to calculate hiring, termination, headcount, and attrition metrics
- **Optional columns**: Useful for segmentation or additional insights
- **Excluded Columns**: Not relevant to this project and removed to reduce noise

| **Required Columns** | **Optional Columns** | **Excluded Columns**            |
|:---------------------|:---------------------|:-------------------------------|
| EmpID                | Title                | EmployeeClassificationType     |
| StartDate            | Division             | TerminationDescription         |
| ExitDate             | PayZone              | JobFunctionDescription         |
| BusinessUnit         | DOB                  | Supervisor                     |
| DepartmentType       | GenderCode           | State                          |
| EmployeeStatus       | RaceDesc             | MaritalDesc                    |
| EmployeeType         | LocationCode         | Performance Score              |
| TerminationType      | FirstName            | Current Employee Rating        |
|                      | LastName             |                                |
|                      | ADEmail              |                                |

In [3]:
# Make a working copy
df = df_raw.copy()

# Define column groups
required_cols = [
    'EmpID', 'StartDate', 'ExitDate',
    'BusinessUnit', 'DepartmentType',
    'EmployeeStatus', 'EmployeeType',
    'TerminationType'
]

optional_cols = [
    'Title', 'Division', 'PayZone',
    'DOB', 'GenderCode', 'RaceDesc', 'LocationCode',
    'FirstName', 'LastName', 'ADEmail'
]

drop_cols = [
    'EmployeeClassificationType', 'TerminationDescription', 'JobFunctionDescription',
    'Supervisor', 'State', 'MaritalDesc',
    'Performance Score', 'Current Employee Rating'
]

# Drop unnecessary columns
df = df.drop(columns=[c for c in drop_cols if c in df.columns])

# Keep only required + optional columns
keep_cols = [c for c in (required_cols + optional_cols) if c in df.columns]
df = df[keep_cols]

print("Columns retained for analysis:")
print(df.columns.tolist())
print("\nShape:", df.shape)

Columns retained for analysis:
['EmpID', 'StartDate', 'ExitDate', 'BusinessUnit', 'DepartmentType', 'EmployeeStatus', 'EmployeeType', 'TerminationType', 'Title', 'Division', 'PayZone', 'DOB', 'GenderCode', 'RaceDesc', 'LocationCode', 'FirstName', 'LastName', 'ADEmail']

Shape: (3000, 18)


# 4. Data Quality Audit & Cleaning

In this step, I performed a combined data quality audit and cleaning process. Each issue is first validated (audit) and then immediately corrected (cleaning),
following the same criteria used during the Excel-based review.

### 🗂️ Audit Checklist
| Check Category | What I Validate | Python Method |
|----------------|------------------|---------------|
| **1. Date Validation** | Convert to datetime, validate StartDate/ExitDate logic | `to_datetime`, `DateOffset`, logical filtering |
| **2. Categorical Columns Inspection & Standardization** | Check label consistency and remove whitespace | `unique()`, `str.strip()` |
| **3. Unique ID Check** | EmpID and names/emails not duplicated | `duplicated()` |
| **4. Unrealistic Values** | DOB outliers (valid age 18–70) | `to_datetime`, age-based filtering |
| **5. Cross-field Consistency** | EmployeeStatus ↔ ExitDate ↔ TerminationType alignment | logical conditions, boolean masking |
| **6. Derived Variables** | IsActive, AttritionFlag, TenureDays | rule-based calculations |


## 4-1. Date Validation
The date fields (`StartDate`, `ExitDate`, `DOB`) were stored in mixed formats, so each field was converted to `datetime` for accurate age, tenure, and employment timeline calculations.

To mirror how Monthly Workforce Snapshots are handled in real HR analytics, all employment dates were shifted by **+2 years and 3 months**, aligning the dataset to **November 2025** and matching the timeline of this project. This preserves all original employment sequences while providing a consistent reference point for downstream metrics such as tenure, age, employment status, and attrition.

### ✔️ Steps Performed
1. Converted all date fields to `datetime` using explicit formats.  
2. Applied a **+2 years, +3 months synthetic shift** to simulate a snapshot as of **November 2025**.  
3. Validated employment timelines by identifying:  
   - `StartDate > ExitDate` (invalid)  
   - `StartDate == ExitDate` (same-day hire/termination)

In [4]:
# 1) Convert raw date strings to datetime
df['StartDate'] = pd.to_datetime(df['StartDate'], format="%d-%b-%y", errors="coerce")
df['ExitDate']  = pd.to_datetime(df['ExitDate'],  format="%d-%b-%y", errors="coerce")
df['DOB']       = pd.to_datetime(df['DOB'],       format="%d-%m-%Y", errors="coerce")

df[['StartDate', 'ExitDate', 'DOB']].head()
df[['StartDate', 'ExitDate', 'DOB']].dtypes

StartDate    datetime64[ns]
ExitDate     datetime64[ns]
DOB          datetime64[ns]
dtype: object

In [5]:
# 2) Synthetic date shift: +2 years, +3 months (simulate Nov 2025 snapshot)

# Before shift (date only)
before_start_min = df['StartDate'].min().date()
before_start_max = df['StartDate'].max().date()
before_exit_min  = df['ExitDate'].min().date()
before_exit_max  = df['ExitDate'].max().date()

# Apply shift (for both StartDate & ExitDate)
shift = pd.DateOffset(years=2, months=3)
for col in ['StartDate', 'ExitDate']:
    df[col] = df[col] + shift

# After shift (date only)
after_start_min = df['StartDate'].min().date()
after_start_max = df['StartDate'].max().date()
after_exit_min  = df['ExitDate'].min().date()
after_exit_max  = df['ExitDate'].max().date()

# Create StartDate comparison table
startdate_comparison = pd.DataFrame({
    "StartDate (min)": [before_start_min, after_start_min],
    "StartDate (max)": [before_start_max, after_start_max]
}, index=["Before Shift", "After Shift"])

# Create ExitDate comparison table
exitdate_comparison = pd.DataFrame({
    "ExitDate (min)": [before_exit_min, after_exit_min],
    "ExitDate (max)": [before_exit_max, after_exit_max]
}, index=["Before Shift", "After Shift"])

startdate_comparison, exitdate_comparison

(             StartDate (min) StartDate (max)
 Before Shift      2018-08-07      2023-08-06
 After Shift       2020-11-07      2025-11-06,
              ExitDate (min) ExitDate (max)
 Before Shift     2018-11-19     2023-08-06
 After Shift      2021-02-19     2025-11-06)

In [6]:
# 3) Identify invalid and same-day cases
invalid_condition = (
    df['StartDate'].notna() &
    df['ExitDate'].notna() &
    (df['StartDate'] > df['ExitDate'])
)

same_day_condition = (
    df['StartDate'].notna() &
    df['ExitDate'].notna() &
    (df['StartDate'] == df['ExitDate'])
)

# Show counts
print("Invalid (StartDate > ExitDate):", invalid_condition.sum())
print("Same-day Start/Exit:", same_day_condition.sum())

# Display same-day hire/termination cases
print("\nReview of same-day hire/termination cases:")
display(df[same_day_condition][['EmpID', 'StartDate','ExitDate', 'EmployeeStatus', 'TerminationType', 'DOB']])

# Add same-day termination flag
df["SameDayTermination"] = same_day_condition

Invalid (StartDate > ExitDate): 0
Same-day Start/Exit: 6

Review of same-day hire/termination cases:


,EmpID,StartDate,ExitDate,EmployeeStatus,TerminationType,DOB
241,3668,2025-11-06,2025-11-06,Active,Voluntary,1996-05-24
443,3870,2025-06-11,2025-06-11,Active,Resignation,1966-11-23
1357,1784,2025-11-02,2025-11-02,Active,Retirement,1971-03-30
1523,1950,2025-10-21,2025-10-21,Voluntarily Terminated,Involuntary,1958-03-24
1543,1970,2024-07-21,2024-07-21,Active,Retirement,1952-04-17
1639,2066,2025-11-04,2025-11-04,Active,Voluntary,1989-05-25


### 📌 Key Findings
- All date fields were successfully converted to `datetime`.
- Synthetic date shifting aligns the dataset to a **November 2025 snapshot** while preserving original timelines.
- **Same-day hire/termination**: 6 cases (retained and flagged).
- **Invalid timelines (StartDate > ExitDate)**: 0 cases.
- Clean date fields now support accurate tenure, age, and attrition calculations.

## 4-2. Categorical Columns Inspection & Standardization

Most categorical fields in this HR dataset were already clean, with only minor issues such as trailing spaces in `DepartmentType` and `Title`. Light normalization was applied to ensure consistent category labels for grouping and downstream HR logic checks.

### ✔️ Steps Performed
1. Identified all categorical (`object`/`category`) fields.  
2. Reviewed unique values to detect formatting inconsistencies.  
3. Saved a snapshot of raw categorical values for before/after comparison.  
4. Applied minimal cleaning (trim whitespace).  
5. Compared before/after counts to confirm merged labels.  
6. Converted categorical fields to `category` dtype for memory efficiency and faster grouping.

In [7]:
# 1) Identify categorical columns
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
cat_cols

# 2) Inspect unique values (Before cleaning)
for col in cat_cols:
    print(f"\n--- {col} ---")
    print(df[col].unique())


--- BusinessUnit ---
['CCDR' 'EW' 'PL' 'TNS' 'BPC' 'WBL' 'NEL' 'SVG' 'MSC' 'PYZ']

--- DepartmentType ---
['Production       ' 'Sales' 'IT/IS' 'Executive Office'
 'Software Engineering' 'Admin Offices']

--- EmployeeStatus ---
['Active' 'Future Start' 'Voluntarily Terminated' 'Leave of Absence'
 'Terminated for Cause']

--- EmployeeType ---
['Contract' 'Full-Time' 'Part-Time']

--- TerminationType ---
['Unk' 'Involuntary' 'Resignation' 'Retirement' 'Voluntary']

--- Title ---
['Production Technician I' 'Area Sales Manager' 'Production Technician II'
 'IT Support' 'Network Engineer' 'Sr. Network Engineer'
 'Principal Data Architect' 'Enterprise Architect' 'Sr. DBA'
 'Database Administrator' 'Data Analyst' 'Data Analyst ' 'Data Architect'
 'CIO' 'BI Director' 'Sr. Accountant' 'Software Engineering Manager'
 'Software Engineer' 'Shared Services Manager' 'Senior BI Developer'
 'Production Manager' 'President & CEO' 'Administrative Assistant'
 'Accountant I' 'BI Developer' 'Sales Manager' 

In [8]:
# 3) Save snapshot (Before cleaning)
# Columns affected by cleanup
clean_cols = ["DepartmentType", "Title"]

# Save original values for comparison
cat_before = df[clean_cols].copy()

# 4) Apply minimal cleaning
# Strip leading/trailing whitespace
df["DepartmentType"] = df["DepartmentType"].str.strip()
df["Title"] = df["Title"].str.strip()

# 5) Before vs After comparison
def show_clean_diff(col):
    print(f"\n=== Clean Diff — {col} ===")
    
    wrap = lambda x: f"⟦{x}⟧"

    before = cat_before[col].astype(str)
    after = df[col].astype(str)

    before_counts = before.value_counts()
    after_counts = after.value_counts()

    changed_labels = set(before[before != after])

    rows = []

    # Original → Cleaned changes
    for v in changed_labels:
        rows.append({
            col: wrap(v),
            "Before": before_counts.get(v, 0),
            "After": after_counts.get(v, 0)
        })

    # Show merged clean label
    for v in changed_labels:
        merged_to = df.loc[before == v, col].iloc[0]
        if merged_to not in changed_labels:  # Avoid duplicates
            rows.append({
                col: wrap(merged_to),
                "Before": before_counts.get(merged_to, 0),
                "After": after_counts.get(merged_to, 0)
            })

    display(pd.DataFrame(rows))

show_clean_diff("DepartmentType")
show_clean_diff("Title")

# 6) Convert to category dtype
for col in cat_cols:
    df[col] = df[col].astype("category")


=== Clean Diff — DepartmentType ===


,DepartmentType,Before,After
0,⟦Production ⟧,2020,0
1,⟦Production⟧,0,2020



=== Clean Diff — Title ===


,Title,Before,After
0,⟦Data Analyst ⟧,8,0
1,⟦Data Analyst⟧,47,55


### 📌 Key Insights

- Only minor inconsistencies were found, primarily trailing spaces in `DepartmentType` and `Title`.
- After trimming whitespace, duplicate labels were merged correctly without altering category meaning.
- All categorical fields are now standardized and stored as `category` dtype, improving memory usage and ensuring consistent values for downstream analysis.

## 4-3. Unique ID Check

To ensure each employee is uniquely identifiable, the dataset was checked for duplicate values across key identifiers: `EmpID`, full name combinations, and email addresses.

In [9]:
# Check duplicated EmpID
print("Duplicate EmpID:", df['EmpID'].duplicated().sum())

# Create full-name field
df['FullName'] = (
    df['FirstName'].astype(str).str.strip() + "_" +
    df['LastName'].astype(str).str.strip()
)

print("Duplicate FullName:", df['FullName'].duplicated().sum())

# Check duplicated email if available
if 'ADEmail' in df.columns:
    print("Duplicate ADEmail:", df['ADEmail'].duplicated().sum())

# Display duplicated full-name groups
print("\nReview of duplicated FullName records:")
name_dupes = df[df['FullName'].duplicated(keep=False)].sort_values(by='FullName')
display(name_dupes[['EmpID','FirstName','LastName','ADEmail','StartDate','ExitDate']].head(20))

Duplicate EmpID: 0
Duplicate FullName: 2
Duplicate ADEmail: 2

Review of duplicated FullName records:


,EmpID,FirstName,LastName,ADEmail,StartDate,ExitDate
1974,2401,Darien,Young,darien.young@bilearner.com,2025-02-09,2025-04-10
2204,2631,Darien,Young,darien.young@bilearner.com,2023-06-03,NaT
865,1292,Larissa,Warner,larissa.warner@bilearner.com,2021-01-27,NaT
1742,2169,Larissa,Warner,larissa.warner@bilearner.com,2022-10-04,2024-08-11


### 📌 Key Insights

- No duplicated `EmpID` values were found.  
- Duplicate full-name and email entries were identified.
- These will be addressed in subsequent cleaning steps as part of the overall quality assurance process.

## 4-4. Realistic DOB (18–70 years)

To ensure that workforce metrics are based on realistic age values, I derived an `Age` column from `DOB` and applied a realistic age range filter (18 ≤ Age < 70). Employees outside this range are likely due to data entry or export errors (e.g., wrong year, default values), so they are treated as DOB outliers and removed from the analytical dataset.

This step prevents unrealistic ages from distorting headcount, tenure, and attrition insights. It also resolves previously detected duplicate name/email records that were tied to invalid DOB values.

In [10]:
# 1) Ensure DOB is parsed correctly (DD-MM-YYYY, day-first)
df['DOB'] = pd.to_datetime(df['DOB'], dayfirst=True, errors='coerce')

# 2) Compute Age using fixed reference date
ref_date = analysis_date  
df['Age'] = ((ref_date - df['DOB']) / pd.Timedelta(days=365.25)).round(1)

print("Age distribution:")
print(df['Age'].describe())

Age distribution:
count    3000.000000
mean       54.210467
std        17.688106
min        24.400000
25%        38.200000
50%        53.900000
75%        69.500000
max        84.300000
Name: Age, dtype: float64


In [11]:
# 3) Define realistic vs outlier age masks
age = df['Age']
realistic_mask = age.notna() & (age >= 18) & (age < 70)
dob_outliers_mask = age.notna() & ~realistic_mask

# 4) Summary before filtering
total = len(df)
realistic_count = realistic_mask.sum()
outliers_count = dob_outliers_mask.sum()

print(f"Total employees: {total}")
print(f"Realistic DOB (18–70): {realistic_count} ({realistic_count/total*100:.1f}%)")
print(f"Unrealistic DOB (<18 or ≥70): {outliers_count} ({outliers_count/total*100:.1f}%)")

print("\nSample DOB outliers:")
display(df[dob_outliers_mask][['EmpID','FirstName','LastName','DOB','Age']].head())

Total employees: 3000
Realistic DOB (18–70): 2280 (76.0%)
Unrealistic DOB (<18 or ≥70): 720 (24.0%)

Sample DOB outliers:


,EmpID,FirstName,LastName,DOB,Age
5,3432,Maruk,Fraval,1949-04-03,76.6
6,3433,Latia,Costa,1942-07-01,83.4
9,3436,Joseph,Martins,1949-11-11,76.0
11,3438,Dheepa,Nguyen,1948-04-06,77.6
13,3440,Xana,Potts,1951-11-06,74.1


In [12]:
# 5) Check duplicates before filtering
dup_fullname_before = df['FullName'].duplicated().sum() if 'FullName' in df.columns else 0
dup_email_before = df['ADEmail'].duplicated().sum() if 'ADEmail' in df.columns else 0

# 6) Filter unrealistic DOBs
df = df[realistic_mask].reset_index(drop=True)

# 7) Check duplicates after filtering
dup_fullname_after = df['FullName'].duplicated().sum() if 'FullName' in df.columns else 0
dup_email_after = df['ADEmail'].duplicated().sum() if 'ADEmail' in df.columns else 0

print("\nDuplicate Check (Before → After DOB Filtering)")
print(f"- FullName: {dup_fullname_before} → {dup_fullname_after}")
print(f"- ADEmail:  {dup_email_before} → {dup_email_after}")
print("\nCleaned dataset shape:", df.shape)


Duplicate Check (Before → After DOB Filtering)
- FullName: 2 → 0
- ADEmail:  2 → 0

Cleaned dataset shape: (2280, 21)


### 📌 Key Insights

- The Age distribution indicated significant DOB inconsistencies, including values as high as 84 years, suggesting data entry or export errors.
- Approximately **76%** of employees fell within the realistic 18–70 age range, while the remaining **24%** were excluded to prevent distortion in tenure, headcount, and attrition metrics.
- After removing these DOB outliers, all previously identified duplicate `FullName` and `ADEmail` records were resolved, confirming that the duplicates originated from invalid DOB values.
- The cleaned dataset now reflects only validated and realistic DOB entries, providing a sound basis for subsequent workforce analysis and EDA.

## 4-5. Cross-field consistency

This step validates whether `EmployeeStatus`, `ExitDate`, and `TerminationType` follow standard HR logic. Records that violate these rules below are corrected or standardized to ensure reliable attrition, tenure, and headcount reporting. 

### HR-consistent Mapping (Expected Relationships)
| EmployeeStatus                       | ExitDate   | TerminationType                      | HR Logic                                   |
| ------------------------------------ | ---------- | ------------------------------------ | ------------------------------------------ |
| **1. Active / Leave of Absence / Future Start** | Not filled | Unk                            | Employee is still employed                 |
| **2. Voluntarily Terminated**           | Filled     | Voluntary / Resignation              | Employee resigned voluntarily              |
| **3. Terminated for Cause**         | Filled     | Involuntary                          | Employer-driven termination   |
| **4. Retired**                          | Filled     | Retirement                           | Exit due to retirement                     |

### ✔️ Steps Performed
1. Detect inconsistencies.
2. Correct misaligned records.
3. Confirm that all rows conform to HR-consistent mappings after cleaning.

In [13]:
# Define HR logic groups (align with dataset labels)
active_status = ['Active', 'Future Start', 'Leave of Absence']
vol_status    = ['Voluntarily Terminated']
invol_status  = ['Terminated for Cause']
ret_status    = ['Retired']

active_types      = ['Unk']
voluntary_types   = ['Voluntary', 'Resignation']
involuntary_types = ['Involuntary']
retirement_types  = ['Retirement']

# Ensure "Retired" category exists
df["EmployeeStatus"] = df["EmployeeStatus"].cat.add_categories(["Retired"])

In [14]:
# 1) Active / Leave of Absence / Future Start
# ==============================
# Expected:
# - ExitDate: empty
# - TerminationType: "Unk"

# Extract Active group
active_df = df[df["EmployeeStatus"].isin(active_status)].copy()

# Normal: no ExitDate AND TerminationType == "Unk"
mask_normal = active_df["ExitDate"].isna() & (active_df["TerminationType"] == "Unk")
normal_active = active_df[mask_normal]

# Error A1: ExitDate present (TerminationType irrelevant)
mask_A1 = active_df["ExitDate"].notna()
error_A1 = active_df[mask_A1]

# Error A2: No ExitDate AND TerminationType != "Unk"
mask_A2 = active_df["ExitDate"].isna() & (active_df["TerminationType"] != "Unk")
error_A2 = active_df[mask_A2]

# Error A3: ExitDate present AND TerminationType != "Unk"
mask_A3 = active_df["ExitDate"].notna() & (active_df["TerminationType"] != "Unk")
error_A3 = active_df[mask_A3]

print("Total Active / Leave of Absence / Future Start rows:", len(active_df))
print("✅ Normal rows:", len(normal_active))
print("❌ Error A1 (Active with ExitDate):", len(error_A1))
print("❌ Error A2 (No ExitDate BUT TermType != 'Unk'):", len(error_A2))
print("❌ Error A3 (ExitDate exists AND TermType != 'Unk'):", len(error_A3))

# Check if A1 and A3 overlap fully
print("A1 and A3 identical?", error_A1.equals(error_A3))

Total Active / Leave of Absence / Future Start rows: 1986
✅ Normal rows: 1113
❌ Error A1 (Active with ExitDate): 873
❌ Error A2 (No ExitDate BUT TermType != 'Unk'): 0
❌ Error A3 (ExitDate exists AND TermType != 'Unk'): 873
A1 and A3 identical? True


In [15]:
# Cleaning: reclassify Active rows that actually have an ExitDate into proper terminated buckets
mask_wrong_active = (
    df["EmployeeStatus"].isin(active_status)
    & df["ExitDate"].notna()
    & (df["TerminationType"] != "Unk")
)

vol_mask = mask_wrong_active & df["TerminationType"].isin(["Voluntary", "Resignation"])
invol_mask = mask_wrong_active & (df["TerminationType"] == "Involuntary")
ret_mask = mask_wrong_active & (df["TerminationType"] == "Retirement")

df.loc[vol_mask, "EmployeeStatus"] = "Voluntarily Terminated"
df.loc[invol_mask, "EmployeeStatus"] = "Terminated for Cause"
df.loc[ret_mask, "EmployeeStatus"] = "Retired"

# Re-validate Active group after cleaning
active_df_clean = df[df["EmployeeStatus"].isin(active_status)].copy()

mask_normal_clean = active_df_clean["ExitDate"].isna() & (active_df_clean["TerminationType"] == "Unk")
mask_A1_clean = active_df_clean["ExitDate"].notna()
mask_A2_clean = active_df_clean["ExitDate"].isna() & (active_df_clean["TerminationType"] != "Unk")
mask_A3_clean = active_df_clean["ExitDate"].notna() & (active_df_clean["TerminationType"] != "Unk")

print("After cleaning – Active / Leave of Absence / Future Start rows:", len(active_df_clean))
print("✅ Normal rows:", mask_normal_clean.sum())
print("❌ Error A1 (Active with ExitDate):", mask_A1_clean.sum())
print("❌ Error A2 (No ExitDate BUT TermType != 'Unk'):", mask_A2_clean.sum())
print("❌ Error A3 (ExitDate exists AND TermType != 'Unk'):", mask_A3_clean.sum())

After cleaning – Active / Leave of Absence / Future Start rows: 1113
✅ Normal rows: 1113
❌ Error A1 (Active with ExitDate): 0
❌ Error A2 (No ExitDate BUT TermType != 'Unk'): 0
❌ Error A3 (ExitDate exists AND TermType != 'Unk'): 0


In [16]:
# 2) Voluntarily Terminated
# ==============================
# Expected:
# - ExitDate: filled
# - TerminationType ∈ {"Voluntary", "Resignation"}

vol_df = df[df["EmployeeStatus"].isin(vol_status)].copy()

# Normal: ExitDate filled AND TerminationType in Voluntary/Resignation
mask_V_normal = vol_df["ExitDate"].notna() & vol_df["TerminationType"].isin(voluntary_types)
normal_vol = vol_df[mask_V_normal]

# Error V1: missing ExitDate
mask_V1 = vol_df["ExitDate"].isna()
error_V1 = vol_df[mask_V1]

# Error V2: wrong TerminationType
mask_V2 = ~vol_df["TerminationType"].isin(voluntary_types)
error_V2 = vol_df[mask_V2]

# Error V3: both missing ExitDate AND wrong TerminationType
mask_V3 = vol_df["ExitDate"].isna() & (~vol_df["TerminationType"].isin(voluntary_types))
error_V3 = vol_df[mask_V3]

print("Total 'Voluntarily Terminated' rows:", len(vol_df))
print("✅ Normal V (ExitDate filled & TermType in Vol/Res):", len(normal_vol))
print("❌ Error V1 (VolTerm but no ExitDate):", len(error_V1))
print("❌ Error V2 (VolTerm but wrong TermType):", len(error_V2))
print("❌ Error V3 (VolTerm, no ExitDate & wrong TermType):", len(error_V3))

Total 'Voluntarily Terminated' rows: 696
✅ Normal V (ExitDate filled & TermType in Vol/Res): 565
❌ Error V1 (VolTerm but no ExitDate): 0
❌ Error V2 (VolTerm but wrong TermType): 131
❌ Error V3 (VolTerm, no ExitDate & wrong TermType): 0


In [17]:
# Cleaning: standardize TerminationType for VolTerm rows
mask_vol_status = df["EmployeeStatus"] == "Voluntarily Terminated"
mask_wrong_vol_type = mask_vol_status & (~df["TerminationType"].isin(voluntary_types))
df.loc[mask_wrong_vol_type, "TerminationType"] = "Voluntary"

# Re-check VolTerm after cleaning
vol_df_clean = df[df["EmployeeStatus"] == "Voluntarily Terminated"].copy()

mask_V_normal_clean = (
    vol_df_clean["ExitDate"].notna() &
    vol_df_clean["TerminationType"].isin(voluntary_types)
)
mask_V1_clean = vol_df_clean["ExitDate"].isna()
mask_V2_clean = ~vol_df_clean["TerminationType"].isin(voluntary_types)
mask_V3_clean = vol_df_clean["ExitDate"].isna() & (~vol_df_clean["TerminationType"].isin(voluntary_types))

print("After cleaning – 'Voluntarily Terminated' rows:", len(vol_df_clean))
print("✅ Normal V (ExitDate filled & TermType in Vol/Res):", mask_V_normal_clean.sum())
print("❌ Error V1 (VolTerm but no ExitDate):", mask_V1_clean.sum())
print("❌ Error V2 (VolTerm but wrong TermType):", mask_V2_clean.sum())
print("❌ Error V3 (VolTerm, no ExitDate & wrong TermType):", mask_V3_clean.sum())

After cleaning – 'Voluntarily Terminated' rows: 696
✅ Normal V (ExitDate filled & TermType in Vol/Res): 696
❌ Error V1 (VolTerm but no ExitDate): 0
❌ Error V2 (VolTerm but wrong TermType): 0
❌ Error V3 (VolTerm, no ExitDate & wrong TermType): 0


In [18]:
# 3) Involuntarily Terminated
# ==============================
# Expected:
# - ExitDate: filled
# - TerminationType = "Involuntary"

invol_df = df[df["EmployeeStatus"].isin(invol_status)].copy()

# Normal: ExitDate filled AND TerminationType == "Involuntary"
mask_I_normal = invol_df["ExitDate"].notna() & invol_df["TerminationType"].isin(involuntary_types)
normal_invol = invol_df[mask_I_normal]

# Error I1: missing ExitDate
mask_I1 = invol_df["ExitDate"].isna()
error_I1 = invol_df[mask_I1]

# Error I2: wrong TerminationType
mask_I2 = ~invol_df["TerminationType"].isin(involuntary_types)
error_I2 = invol_df[mask_I2]

# Error I3: missing ExitDate + wrong TerminationType
mask_I3 = invol_df["ExitDate"].isna() & (~invol_df["TerminationType"].isin(involuntary_types))
error_I3 = invol_df[mask_I3]

print("\nBefore cleaning – Involuntary statuses rows:", len(invol_df))
print("✅ Normal I (ExitDate filled & TermType = Involuntary):", len(normal_invol))
print("❌ Error I1 (InvolTerm but no ExitDate):", len(error_I1))
print("❌ Error I2 (InvolTerm but wrong TermType):", len(error_I2))
print("❌ Error I3 (InvolTerm, no ExitDate & wrong TermType):", len(error_I3))


Before cleaning – Involuntary statuses rows: 254
✅ Normal I (ExitDate filled & TermType = Involuntary): 222
❌ Error I1 (InvolTerm but no ExitDate): 0
❌ Error I2 (InvolTerm but wrong TermType): 32
❌ Error I3 (InvolTerm, no ExitDate & wrong TermType): 0


In [19]:
# Cleaning: standardize Involuntary termination types
mask_invol_status = df["EmployeeStatus"].isin(invol_status)
mask_wrong_invol_type = mask_invol_status & (~df["TerminationType"].isin(involuntary_types))
df.loc[mask_wrong_invol_type, "TerminationType"] = "Involuntary"

# Re-check InvolTerm after cleaning
invol_df_clean = df[df["EmployeeStatus"].isin(invol_status)].copy()

mask_I_normal_clean = (
    invol_df_clean["ExitDate"].notna() &
    invol_df_clean["TerminationType"].isin(involuntary_types)
)
mask_I1_clean = invol_df_clean["ExitDate"].isna()
mask_I2_clean = ~invol_df_clean["TerminationType"].isin(involuntary_types)
mask_I3_clean = invol_df_clean["ExitDate"].isna() & (~invol_df_clean["TerminationType"].isin(involuntary_types))

print("\nAfter cleaning – Involuntary statuses rows:", len(invol_df_clean))
print("✅ Normal I (ExitDate filled & TermType = Involuntary):", mask_I_normal_clean.sum())
print("❌ Error I1 (InvolTerm but no ExitDate):", mask_I1_clean.sum())
print("❌ Error I2 (InvolTerm but wrong TermType):", mask_I2_clean.sum())
print("❌ Error I3 (InvolTerm, no ExitDate & wrong TermType):", mask_I3_clean.sum())


After cleaning – Involuntary statuses rows: 254
✅ Normal I (ExitDate filled & TermType = Involuntary): 254
❌ Error I1 (InvolTerm but no ExitDate): 0
❌ Error I2 (InvolTerm but wrong TermType): 0
❌ Error I3 (InvolTerm, no ExitDate & wrong TermType): 0


In [20]:
# 4) Retired
# ==============================
# Expected:
# - ExitDate: filled
# - TerminationType = "Retirement"

ret_df = df[df["EmployeeStatus"] == "Retired"].copy()

mask_R_expected = ret_df["ExitDate"].notna() & (ret_df["TerminationType"] == "Retirement")
normal_ret = ret_df[mask_R_expected]

mask_R1 = ret_df["ExitDate"].isna()
error_R1 = ret_df[mask_R1]

mask_R2 = ret_df["TerminationType"] != "Retirement"
error_R2 = ret_df[mask_R2]

mask_R3 = mask_R1 & mask_R2
error_R3 = ret_df[mask_R3]

print("\nBefore cleaning – EmployeeStatus = 'Retired' rows:", len(ret_df))
print("✅ Normal R (ExitDate filled & TerminationType = 'Retirement'):", len(normal_ret))
print("❌ Error R1 (Retired but no ExitDate):", len(error_R1))
print("❌ Error R2 (Retired but TerminationType != 'Retirement'):", len(error_R2))
print("❌ Error R3 (Retired, no ExitDate AND wrong TerminationType):", len(error_R3))


Before cleaning – EmployeeStatus = 'Retired' rows: 217
✅ Normal R (ExitDate filled & TerminationType = 'Retirement'): 217
❌ Error R1 (Retired but no ExitDate): 0
❌ Error R2 (Retired but TerminationType != 'Retirement'): 0
❌ Error R3 (Retired, no ExitDate AND wrong TerminationType): 0


In [21]:
# 5) Final HR-consistent mapping summary

mask_active_group = df["EmployeeStatus"].isin(active_status)
mask_vol_group    = df["EmployeeStatus"].isin(vol_status)
mask_invol_group  = df["EmployeeStatus"].isin(invol_status)
mask_ret_group    = df["EmployeeStatus"].isin(ret_status)

# HR-consistent rules
mask_rule1_valid = (
    mask_active_group &
    df["ExitDate"].isna() &
    (df["TerminationType"] == "Unk")
)

mask_rule2_valid = (
    mask_vol_group &
    df["ExitDate"].notna() &
    df["TerminationType"].isin(["Voluntary", "Resignation"])
)

mask_rule3_valid = (
    mask_invol_group &
    df["ExitDate"].notna() &
    (df["TerminationType"] == "Involuntary")
)

mask_rule4_valid = (
    mask_ret_group &
    df["ExitDate"].notna() &
    (df["TerminationType"] == "Retirement")
)

summary_data = [
    {
        "Rule": "1. Active / LOA / Future Start",
        "EmployeeStatus group": "Active, Leave of Absence, Future Start",
        "Total rows": int(mask_active_group.sum()),
        "Valid (HR-consistent)": int(mask_rule1_valid.sum()),
        "Invalid (violations)": int(mask_active_group.sum() - mask_rule1_valid.sum()),
    },
    {
        "Rule": "2. Voluntarily Terminated",
        "EmployeeStatus group": "Voluntarily Terminated",
        "Total rows": int(mask_vol_group.sum()),
        "Valid (HR-consistent)": int(mask_rule2_valid.sum()),
        "Invalid (violations)": int(mask_vol_group.sum() - mask_rule2_valid.sum()),
    },
    {
        "Rule": "3. Involuntarily Terminated",
        "EmployeeStatus group": "Terminated for Cause",
        "Total rows": int(mask_invol_group.sum()),
        "Valid (HR-consistent)": int(mask_rule3_valid.sum()),
        "Invalid (violations)": int(mask_invol_group.sum() - mask_rule3_valid.sum()),
    },
    {
        "Rule": "4. Retired",
        "EmployeeStatus group": "Retired",
        "Total rows": int(mask_ret_group.sum()),
        "Valid (HR-consistent)": int(mask_rule4_valid.sum()),
        "Invalid (violations)": int(mask_ret_group.sum() - mask_rule4_valid.sum()),
    },
]

summary_df = pd.DataFrame(summary_data)

print("\nHR-consistent Mapping Final Check")
display(summary_df)


HR-consistent Mapping Final Check


,Rule,EmployeeStatus group,Total rows,Valid (HR-consistent),Invalid (violations)
0,1. Active / LOA / Future Start,"Active, Leave of Absence, Future Start",1113,1113,0
1,2. Voluntarily Terminated,Voluntarily Terminated,696,696,0
2,3. Involuntarily Terminated,Terminated for Cause,254,254,0
3,4. Retired,Retired,217,217,0


### 📌 Key Insights

- By systematically classifying errors (A1–A3, V1–V3, I1–I3, R1–R3) and applying HR-consistent rules, misaligned records were either corrected or structurally fixed.
    - Records labeled as **Active but containing an ExitDate were reclassified into the correct termination groups** (Voluntary, Involuntary, Retired), ensuring statuses accurately reflect employment history.
    - **TerminationType values were standardized** using EmployeeStatus as the source of truth, correcting cases where the type did not match the employee’s status.
- After cleaning, all four groups (Active, Voluntary, Involuntary, Retired) fully matched the expected HR rules, with no remaining inconsistencies.
- The dataset now reflects a **clean, HR-consistent employment lifecycle**, supporting accurate attrition, tenure, and headcount analysis.

## 4-6. Derived Variables

With the core cleaning steps complete, I generated several derived variables that will support the upcoming EDA and workforce metrics.

**Derived in this step**
- `IsActive`: identifies employees currently on payroll
- `AttritionFlag`: 1 for employees who have left the company, 0 otherwise
- `TenureDays`: number of days employed (Start→Exit for leavers, Start→RefDate for active employees)

**Created earlier in previous steps**
- `SameDayTermination`(from 4.1 Date Validation)
- `FullName`(from 4.3 Unique ID Check)
- `Age`(from 4.4 Realistic DOB)

In [22]:
# IsActive flag
active_status = ["Active", "Leave of Absence", "Future Start"]

df["IsActive"] = (
    df["EmployeeStatus"].isin(active_status)
    & df["ExitDate"].isna()
)

df["IsActive"] = df["IsActive"].astype(bool)

print("Active employees (IsActive = True):", df["IsActive"].sum())

# AttritionFlag
terminated_status = [
    "Voluntarily Terminated",
    "Terminated for Cause",
    "Retired",
]

df["AttritionFlag"] = np.where(
    df["EmployeeStatus"].isin(terminated_status) & df["ExitDate"].notna(),
    1,
    0,
).astype(int)

print("Employees with AttritionFlag = 1:", df["AttritionFlag"].sum())

Active employees (IsActive = True): 1113
Employees with AttritionFlag = 1: 1167


In [23]:
# SameDayTermination flag check
print("Same-day terminations:", df["SameDayTermination"].sum())

# TenureDays (includes 0-day separations)
today = analysis_date.normalize()

df["TenureDays"] = np.nan

# For terminated employees
mask_has_exit = df["StartDate"].notna() & df["ExitDate"].notna()
df.loc[mask_has_exit, "TenureDays"] = (
    df.loc[mask_has_exit, "ExitDate"] - df.loc[mask_has_exit, "StartDate"]
).dt.days

# For active employees
mask_active_no_exit = df["StartDate"].notna() & df["ExitDate"].isna()
df.loc[mask_active_no_exit, "TenureDays"] = (
    today - df.loc[mask_active_no_exit, "StartDate"]
).dt.days

# Same-day terminations = 0 tenure
df.loc[df["SameDayTermination"], "TenureDays"] = 0

df["TenureDays"] = df["TenureDays"].astype("Int64")

print("Zero-day separations:", (df["TenureDays"] == 0).sum())


# Negative tenure check (sanity check)
negative_tenure_count = (df["TenureDays"] < 0).sum()
print("\nNegative tenure count:", negative_tenure_count)

print("\nShape:", df.shape)

Same-day terminations: 5
Zero-day separations: 5

Negative tenure count: 0

Shape: (2280, 24)


### 📌 Key Insights

- Derived variables were created only after the core cleaning steps were completed, ensuring they accurately reflect the cleaned and corrected employment lifecycle.
- **IsActive (1113) and AttritionFlag = 1 (1167) sum to the total dataset size (2280)**, confirming all employees are consistently classified with no lifecycle gaps.
- Before cleaning, there were 6 same-day hire/termination cases. During debugging, I confirmed that the reduced count (5) was **directly caused by the Realistic DOB cleaning rule**, which removed one employee aged 70+. This expected change highlights the importance of revalidating derived metrics after all cleaning steps.
- No negative-tenure records remain, confirming clean and valid employment timelines.
- The final cleaned dataset now includes **24 well-defined columns**, ready for a reliable workforce EDA.

# 5. Finalize Column Structure

Organizing columns ensures a clean and consistent analysis workflow.

In [24]:
# Check current columns
df.columns

Index(['EmpID', 'StartDate', 'ExitDate', 'BusinessUnit', 'DepartmentType',
       'EmployeeStatus', 'EmployeeType', 'TerminationType', 'Title',
       'Division', 'PayZone', 'DOB', 'GenderCode', 'RaceDesc', 'LocationCode',
       'FirstName', 'LastName', 'ADEmail', 'SameDayTermination', 'FullName',
       'Age', 'IsActive', 'AttritionFlag', 'TenureDays'],
      dtype='object')

In [25]:
# Define the desired column order
cols_order = [
    # --- Identity ---
    'EmpID', 'FirstName', 'LastName', 'NameCombo', 'ADEmail',

    # --- Employment Dates & Status ---
    'StartDate', 'ExitDate', 'EmployeeStatus', 'IsActive', 'AttritionFlag',

    # --- Derived Metrics ---
    'TenureDays', 'SameDayTermination', 'Age',

    # --- Job / Org Details ---
    'Title', 'BusinessUnit', 'DepartmentType', 'Division',
    'EmployeeType', 'PayZone', 'TerminationType',

    # --- Demographics ---
    'DOB', 'GenderCode', 'RaceDesc', 'LocationCode'
]
# Keep only the columns that actually exist in the dataframe
cols_order = [c for c in cols_order if c in df.columns]

# Reorder dataframe columns
df = df[cols_order]

df.head()

,EmpID,FirstName,LastName,ADEmail,StartDate,ExitDate,EmployeeStatus,IsActive,AttritionFlag,TenureDays,SameDayTermination,Age,Title,BusinessUnit,DepartmentType,Division,EmployeeType,PayZone,TerminationType,DOB,GenderCode,RaceDesc,LocationCode
0,3427,Uriah,Bridges,uriah.bridges@bilearner.com,2021-12-20,NaT,Active,True,0,1437,False,56.1,Production Technician I,CCDR,Production,Finance & Accounting,Contract,Zone C,Unk,1969-10-07,Female,White,34904
1,3428,Paula,Small,paula.small@bilearner.com,2025-05-11,NaT,Active,True,0,199,False,60.2,Production Technician I,EW,Production,Aerial,Contract,Zone A,Unk,1965-08-30,Male,Hispanic,6593
2,3429,Edward,Buck,edward.buck@bilearner.com,2021-03-10,NaT,Active,True,0,1722,False,34.1,Area Sales Manager,PL,Sales,General - Sga,Full-Time,Zone B,Unk,1991-10-06,Male,Hispanic,2330
3,3430,Michael,Riordan,michael.riordan@bilearner.com,2023-09-21,NaT,Active,True,0,797,False,27.6,Area Sales Manager,CCDR,Sales,Finance & Accounting,Contract,Zone A,Unk,1998-04-04,Male,Other,58782
4,3431,Jasmine,Onque,jasmine.onque@bilearner.com,2021-09-29,NaT,Active,True,0,1519,False,56.2,Area Sales Manager,TNS,Sales,General - Con,Contract,Zone A,Unk,1969-08-29,Female,Other,33174


# 6. Export Cleaned Dataset

This cleaned file will be used in **Notebook 2 – HR Workforce analysis**.

In [26]:
df.to_csv("cleaned_employee_data.csv", index=False)
print("Cleaned dataset saved!")

Cleaned dataset saved!
